# Directions

IS 675: Data Mining
Summer 2019
Homework Assignment 1 (Due June 16, 2019, midnight EST)


Use the iris dataset attached to this assignment.   (20 points each)

1. Load the iris data into a matrix. Note that you can modify the `file2matrix()` function in the book to do this. 
1. Note that the data is ordered by the class, so it needs to be shuffled. Import numpy  and use the `random.shuffle()` function to shuffle the data. \[Hint: `np.random.shuffle(irisDataMat)`]
1. Divide the data such that the data and class labels are stored in separate arrays.
1. Divide the data into four matrices: training data, training labels, test data, test labels. Training data should be the first 100 rows and test data should contain the last 50 rows.
1. Use the `classify0()` and `datingClassTest()` functions shown in the book to test the classification accuracy. Modify the `datingClassTest()` appropriately with a meaningful name and code to suit the iris dataset.

## Thanks 
Thanks to Peter Harrington for posting many of the functions used here. They can be found at https://github.com/pbharrin/machinelearninginaction/blob/master/Ch02/kNN.py


In [131]:
import numpy as np
import operator
from numpy import zeros, tile, shape

# 1. Load the iris data into a matrix. 

Note that you can modify the `file2matrix()` function in the book to do this.

In [81]:
iris_dictionary={'I. setosa':1, 'I. versicolor':2, 'I. virginica':3}


def file2matrix(filename):
    fr = open(filename)
    arrayOLines = fr.readlines()
    numberOfLines = len(arrayOLines)            #get the number of lines in the file
    numberOfColumns = len(arrayOLines[0].split(',')) - 1
    returnMat = zeros((numberOfLines,numberOfColumns))        #prepare matrix to return
    classLabelVector = []                       #prepare labels return   
    index = 0
    for line in arrayOLines:
        line = line.strip()
        listFromLine = line.split(',')
        returnMat[index,:] = listFromLine[0:numberOfColumns]
        if(listFromLine[-1].isdigit()):
            classLabelVector.append(int(listFromLine[-1]))
        else:
            classLabelVector.append(iris_dictionary.get(listFromLine[-1]))
        index += 1
    return returnMat, classLabelVector

file_name = 'iris_data.txt'
irisDataMat, irisDataClass = file2matrix(file_name)
irisDataClass = np.asarray(irisDataClass)
print(irisDataMat[:5], '\n', irisDataMat.shape, '\n')
print(irisDataClass[:5], '\n', irisDataClass.shape)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]] 
 (150, 4) 

[1 1 1 1 1] 
 (150,)


## 2. Shuffle 
Note that the data is ordered by the class, so it needs to be shuffled. Import numpy and use the `random.shuffle()` function to shuffle the data. [Hint: `np.random.shuffle(irisDataMat)`]


In [100]:
randomize = np.arange(len(irisDataMat))
np.random.shuffle(randomize)

irisDataMat = irisDataMat[randomize]
print(irisDataMat[:5], '\n')

irisDataClass = irisDataClass[randomize]
print(irisDataClass[:5])

[[6.  2.2 4.  1. ]
 [6.7 3.  5.2 2.3]
 [5.4 3.4 1.5 0.4]
 [6.1 2.8 4.7 1.2]
 [4.9 2.4 3.3 1. ]] 

[2 1 1 3 1]


## 3. Divide the data such that the data and class labels are stored in separate arrays.


In [99]:
print('data', irisDataMat[:5])
print('\n classes encoded ', irisDataClass[:5])

iris_class_labels_reverse = dict((v, k) for k, v in iris_dictionary.items())
iris_class_labels = [iris_class_labels_reverse[i] for i in irisDataClass]
print('\n class labels', iris_class_labels[:5])

data [[6.3 2.3 4.4 1.3]
 [5.  3.6 1.4 0.2]
 [7.6 3.  6.6 2.1]
 [5.  3.5 1.6 0.6]
 [5.6 2.7 4.2 1.3]]

 class  [2 3 3 2 1]

 class labels ['I. versicolor', 'I. virginica', 'I. virginica', 'I. versicolor', 'I. setosa']


## 4. Divide the data into four matrices: training data, training labels, test data, test labels. Training data should be the first 100 rows and test data should contain the last 50 rows.


In [107]:
training_data = irisDataMat[:100]
test_data = irisDataMat[-50:]
print('training data \n', training_data[:5], '\n shape: ', training_data.shape)
print('\ntest data \n', test_data[:5], '\n shape: ', test_data.shape)

training_labels = iris_class_labels[:100]
test_labels = iris_class_labels[-50:]
print('\ntraining labels \n', training_labels[:5], '\n len: ', len(training_labels))
print('\ntest labels \n', test_labels[:5], '\n len: ', len(test_labels))


training data 
 [[6.  2.2 4.  1. ]
 [6.7 3.  5.2 2.3]
 [5.4 3.4 1.5 0.4]
 [6.1 2.8 4.7 1.2]
 [4.9 2.4 3.3 1. ]] 
 shape:  (100, 4)

test data 
 [[4.8 3.1 1.6 0.2]
 [6.5 3.  5.5 1.8]
 [5.  3.4 1.5 0.2]
 [6.9 3.1 5.1 2.3]
 [4.5 2.3 1.3 0.3]] 
 shape:  (50, 4)

training labels 
 ['I. versicolor', 'I. virginica', 'I. virginica', 'I. versicolor', 'I. setosa'] 
 len:  100

test labels 
 ['I. versicolor', 'I. setosa', 'I. setosa', 'I. versicolor', 'I. versicolor'] 
 len:  50


## 5. Use the `classify0()` and `datingClassTest()` functions shown in the book to test the classification accuracy. Modify the `datingClassTest()` appropriately with a meaningful name and code to suit the iris dataset.

In [120]:
def classify0(inX, dataSet, labels, k):
    dataSetSize = dataSet.shape[0]
    diffMat = tile(inX, (dataSetSize,1)) - dataSet
    sqDiffMat = diffMat**2
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances**0.5
    sortedDistIndicies = distances.argsort()     
    classCount={}          
    for i in range(k):
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel,0) + 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

k = 3
num_correct = 0

for x, y in zip(test_data, test_labels):
    classified_as = classify0(x, training_data, training_labels, k)
    correct = y == classified_as
    if correct:
        num_correct += 1
    print(y, '\t classified as \t', classified_as, '\t is \t', correct)

print('\n number correct: ', num_correct, '/', len(test_data))

I. versicolor 	 classified as 	 I. setosa 	 is 	 False
I. setosa 	 classified as 	 I. virginica 	 is 	 False
I. setosa 	 classified as 	 I. versicolor 	 is 	 False
I. versicolor 	 classified as 	 I. virginica 	 is 	 False
I. versicolor 	 classified as 	 I. virginica 	 is 	 False
I. setosa 	 classified as 	 I. versicolor 	 is 	 False
I. virginica 	 classified as 	 I. setosa 	 is 	 False
I. virginica 	 classified as 	 I. versicolor 	 is 	 False
I. setosa 	 classified as 	 I. versicolor 	 is 	 False
I. virginica 	 classified as 	 I. setosa 	 is 	 False
I. setosa 	 classified as 	 I. versicolor 	 is 	 False
I. setosa 	 classified as 	 I. virginica 	 is 	 False
I. setosa 	 classified as 	 I. versicolor 	 is 	 False
I. virginica 	 classified as 	 I. setosa 	 is 	 False
I. virginica 	 classified as 	 I. versicolor 	 is 	 False
I. virginica 	 classified as 	 I. versicolor 	 is 	 False
I. setosa 	 classified as 	 I. versicolor 	 is 	 False
I. virginica 	 classified as 	 I. virginica 	 is 	 True

In [133]:
def autoNorm(dataSet):
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    normDataSet = zeros(shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - tile(minVals, (m,1))
    normDataSet = normDataSet/tile(ranges, (m,1))   #element wise divide
    return normDataSet, ranges, minVals

def datingClassTest():
    hoRatio = 0.50      #hold out 10%
    datingDataMat,datingLabels = file2matrix(file_name)       #load data setfrom file
    normMat, ranges, minVals = autoNorm(datingDataMat)
    m = normMat.shape[0]
    numTestVecs = int(m*hoRatio)
    errorCount = 0.0
    for i in range(numTestVecs):
        classifierResult = classify0(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],3)
        print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, datingLabels[i]))
        if (classifierResult != datingLabels[i]): errorCount += 1.0
    print("the total error rate is: %f" % (errorCount/float(numTestVecs)))
    print(errorCount)
    
datingClassTest()

the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answer is: 1
the classifier came back with: 2, the real answe